In [ ]:
#Import package
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib
from geopy.distance import geodesic
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from datetime import datetime, timedelta, date
import math

In [ ]:
# - Import data -
aed = pd.read_parquet('aed_locations.parquet.gzip')

In [ ]:
# AED
print(aed.head())
aed_cl=aed.copy()

for column in aed_cl.columns:
    if aed_cl[column].dtype=='object':
        aed_cl[column]=aed_cl[column].str.title().str.strip()
    else:
        aed_cl[column]=aed_cl[column]    

aed_cl.loc[aed_cl['type'].isin(['Appareil fixe','Appareil Fixe','Appareil fixe-Vast apparaat','Vast apparaat','Vast apparaat']),'type']='Fixed'
aed_cl.loc[aed_cl['type'].isin(['Appareil Mobile- Mobiel apparaat','M5066A']),'type']='Mobile'

aed_cl['postal_code']=aed_cl['postal_code'].astype('str').str.replace('.0','')

for index, value in aed.iterrows():
    if aed_cl.iloc[index, 8] in ['J', 'Ja', 'Oui', 'Oui-Ja', 'y', 'Y']:
        aed_cl.loc[index, 'public'] = 'Yes'
    elif aed_cl.iloc[index, 8] in ['Non-Nee', 'N', 'Nee']:
        aed_cl.loc[index, 'public'] = 'No'
    else:
        aed_cl.iloc[index, 8] = 'None'
        
for index, value in aed.iterrows():
    if aed_cl.iloc[index, 9] in ['J', 'Ja', 'Oui', 'Oui-Ja', 'y', 'Y', 'oui via interphone','Accessible par toute personne présente dans l inrfastructure.','Oui-Ja "Niet tijdens activiteiten van de wielervrienden"']:
        aed_cl.loc[index, 'available'] = 'Yes'
    elif aed_cl.iloc[index, 9] in ['Non-Nee', 'N', 'Nee']:
        aed_cl.loc[index, 'available'] = 'No'
    elif aed_cl.iloc[index, 9] in ['09.00 - 17.00', '9.00 - 17.00', 'Nee']:
        aed_cl.loc[index, 'available'] = '09.00 - 17.00'
    elif aed_cl.iloc[index, 9] in ['09u-12u en op aanvraag']:
        aed_cl.loc[index, 'available'] = '09.00 - 12.00'
    elif aed_cl.iloc[index, 9] in ['5h00 à 25h00', 'de 5h00 à 25h00']:
        aed_cl.loc[index, 'available'] = '05.00 - 23.59'
    elif aed_cl.iloc[index, 9] in ['8:00 - 17:00', 'N', 'Nee']:
        aed_cl.loc[index, 'available'] = '8:00 - 17:00'
    elif aed_cl.iloc[index, 9] in ['16u-23u tijdens opening sporthal']:
        aed_cl.loc[index, 'available'] = '16.00 - 23.00'
    elif aed_cl.iloc[index, 9] in ['De 5h30 à 21h30']:
        aed_cl.loc[index, 'available'] = '05.30 - 21.30'
    elif aed_cl.iloc[index, 9] in ['Dispo 24/7 - sauf samedi de 12h à dimanche 20h', 'Dispo. tout le temps sauf du samedi midi au dimanche 21h']:
        aed_cl.loc[index, 'available'] = '00.00 - 23.59'
    elif aed_cl.iloc[index, 9] in ['Heure de bureau en semaine', 'Heures de bureau', 'Horaire d ouverture de la buvette','Pendant heures d ouverture du site',
                                   'Pendant les heures de cours','Tijdens de kantooruren','Tijdens kantooruren','Tijdens openingsuren van het museum','enkel tijdens kantooruren',
                                   'enkel tijdens de kantooruren (8 - 19u)','horaire d ouverture de la pharmacie','indien de site open is, volledig toegangkelijk','kantooruren',
                                   'selon heures d ouverture d Euro-Délices','tijdens de kantooruren','tijdens de kantooruren in de week','tijdens de openingsuren','tijdens de werkuren',
                                   'tijdens onze openingsuren, zie nr. 12','tijdens openinguren sportcentrum','tijdens werkuren','zie rooster','Heures de bureau ']:
        aed_cl.loc[index, 'available'] = 'Working Opening Time'
    elif aed_cl.iloc[index, 9] in ['maandag, dinsdag, donderdag, vrijdag', 'du lundi au vendredi ']: 
        aed_cl.loc[index, 'available'] = 'During Weekdays'
    else:
        aed_cl.iloc[index, 9] = 'None'  

aed_all = pd.merge(aed_cl,postal_codes_subset, left_on = 'postal_code', right_on = 'postcode', how ='left')
for a, b in zip(aed_all.columns, range(0,len(aed_all.columns))):
                print(f'{a}, column number {b}')

aed_subset = aed_all.iloc[:,[0,1,2,3,17,18,19,20,1,4,8,9,10]]
aed_subset.head()

In [ ]:
# postal_codes_subset.to_csv('postal_codes.csv', index=False)
aed_subset.to_csv('aed.csv', index=False)